In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import classification_report


In [6]:
true = pd.read_csv('True.csv')

In [7]:
fake = pd.read_csv('Fake.csv')

In [8]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:
true['label']=1

In [11]:
fake['label']=0

In [12]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [13]:
news = pd.concat([fake , true],axis=0)

In [14]:
news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [15]:
news.tail()

,title,text,subject,date,label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [16]:
news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [17]:
news = news.drop(['title','subject','date'],axis=1)

In [18]:
news.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [19]:
news = news.sample(frac=1)                #Reshuffling

In [20]:
news.head()

,text,label
13318,"POTSDAM, Germany (Reuters) - German police on ...",1
14662,KHARTOUM (Reuters) - A 72-year-old Swiss aid w...,1
6201,"WASHINGTON (Reuters) - Rick Perry, President-e...",1
13844,We all know the defense of children comes dead...,0
9956,(Reuters) - A South Carolina bill to require t...,1


In [21]:
news.reset_index(inplace=True)

In [22]:
news.head()

,index,text,label
0,13318,"POTSDAM, Germany (Reuters) - German police on ...",1
1,14662,KHARTOUM (Reuters) - A 72-year-old Swiss aid w...,1
2,6201,"WASHINGTON (Reuters) - Rick Perry, President-e...",1
3,13844,We all know the defense of children comes dead...,0
4,9956,(Reuters) - A South Carolina bill to require t...,1


In [23]:
news.drop(['index'],axis=1, inplace = True)

In [24]:
news.head()

,text,label
0,"POTSDAM, Germany (Reuters) - German police on ...",1
1,KHARTOUM (Reuters) - A 72-year-old Swiss aid w...,1
2,"WASHINGTON (Reuters) - Rick Perry, President-e...",1
3,We all know the defense of children comes dead...,0
4,(Reuters) - A South Carolina bill to require t...,1


In [2]:
def wordopt(text):
    #convert into lowercase
    text = text.lower()

    # Remove URLs 
    text = re.sub(r'https?://\S+|www\.\S+','',text)

    # Remove HTML tags
    text = re.sub(r'<.*?>','',text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]','',text)

    # Remove digits
    text = re.sub(r'\d','',text)

    # Remove newline character
    text = re.sub(r'\n','',text)

    return text

In [26]:
news['text']= news['text'].apply(wordopt)

In [27]:
news['text']

0        potsdam germany reuters  german police on frid...
1        khartoum reuters  a yearold swiss aid worker w...
2        washington reuters  rick perry presidentelect ...
3        we all know the defense of children comes dead...
4        reuters  a south carolina bill to require tran...
                               ...                        
44893                                                     
44894    washington reuters  republican presidential ca...
44895    it was just announced today that the obama s a...
44896    oh the irony of a terror attack by muslims tak...
44897    moscow reuters  russian foreign minister serge...
Name: text, Length: 44898, dtype: object

In [28]:
x = news['text']
y = news['label']

In [29]:
x

0        potsdam germany reuters  german police on frid...
1        khartoum reuters  a yearold swiss aid worker w...
2        washington reuters  rick perry presidentelect ...
3        we all know the defense of children comes dead...
4        reuters  a south carolina bill to require tran...
                               ...                        
44893                                                     
44894    washington reuters  republican presidential ca...
44895    it was just announced today that the obama s a...
44896    oh the irony of a terror attack by muslims tak...
44897    moscow reuters  russian foreign minister serge...
Name: text, Length: 44898, dtype: object

In [30]:
y

0        1
1        1
2        1
3        0
4        1
        ..
44893    0
44894    1
44895    0
44896    0
44897    1
Name: label, Length: 44898, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [33]:
x_train.shape

(31428,)

In [34]:
x_test.shape

(13470,)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vectorization = TfidfVectorizer()

In [37]:
xv_train = vectorization.fit_transform(x_train)

In [38]:
xv_test = vectorization.transform(x_test)

In [39]:
xv_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6470257 stored elements and shape (31428, 175078)>

In [40]:
xv_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2710555 stored elements and shape (13470, 175078)>

In [41]:
#logistic regression

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
LR = LogisticRegression()

In [44]:
LR.fit(xv_train, y_train)

LogisticRegression()

In [45]:
pred_lr = LR.predict(xv_test)

In [46]:
LR.score(xv_test,y_test)

0.988641425389755

In [47]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7024
           1       0.99      0.99      0.99      6446

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [48]:
from sklearn.tree import DecisionTreeClassifier

In [49]:
DTC = DecisionTreeClassifier()

In [50]:
DTC.fit(xv_train, y_train)

DecisionTreeClassifier()

In [51]:
pred_dtc = DTC.predict(xv_test)

In [52]:
DTC.score(xv_test, y_test)

0.9962880475129918

In [53]:
print(classification_report(y_test, pred_dtc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7024
           1       1.00      1.00      1.00      6446

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
rfc = RandomForestClassifier()

In [56]:
rfc.fit(xv_train, y_train)

RandomForestClassifier()

In [57]:
predict_rfc = rfc.predict(xv_test)

In [58]:
rfc.score(xv_test, y_test)

0.9860430586488493

In [59]:
from sklearn.ensemble import GradientBoostingClassifier


In [60]:
gbc = GradientBoostingClassifier()

In [61]:
gbc.fit(xv_train, y_train)

GradientBoostingClassifier()

In [62]:
pred_gbc = gbc.predict(xv_test)

In [63]:
gbc.score(xv_test, y_test)

0.9953229398663697

In [64]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7024
           1       0.99      1.00      1.00      6446

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [3]:
def output_label(n):
    if n==0:
        return "It is a Fake News"
    elif n==1:
        return "It is a Genuine News"

In [4]:
def manual_testing(news):
    testing_news = {"text": [news]}  # Corrected syntax for defining dictionary
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)  # Assuming 'vectorization' is your vectorizer object
    pred_lr = LR.predict(new_xv_test)
  #pred_dtc = dtc.predict(new_xv_test)
    pred_gbc = gbc.predict(new_xv_test)
    pred_rfc = rfc.predict(new_xv_test)
    return "\n\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(
        output_label(pred_lr[0]), output_label(pred_gbc[0]), output_label(pred_rfc[0]))
    


In [67]:
news_article = str(input())

In [68]:
manual_testing(news_article)

'\n\nLR Prediction: It is a Fake News \nGBC Prediction: It is a Fake News \nRFC Prediction: It is a Fake News'

In [ ]:
news_article = str(input())

In [70]:
manual_testing(news_article)

'\n\nLR Prediction: It is a Fake News \nGBC Prediction: It is a Fake News \nRFC Prediction: It is a Fake News'

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

# Load datasets
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")

# Add labels: 0 = Real, 1 = Fake
true_df["label"] = 0
fake_df["label"] = 1

# Combine datasets
df = pd.concat([true_df, fake_df], axis=0).reset_index(drop=True)

# Select only text and labels
X = df["text"]  # News articles
y = df["label"]  # Labels (0 = Real, 1 = Fake)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Save the trained model and vectorizer
joblib.dump(model, "fake_news_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

print("✅ Model training complete! Saved as fake_news_model.pkl")


FileNotFoundError: [Errno 2] No such file or directory: 'True.csv'

In [72]:
news_article = str(input())

In [73]:
manual_testing(news_article)

'\n\nLR Prediction: It is a Fake News \nGBC Prediction: It is a Fake News \nRFC Prediction: It is a Fake News'

In [14]:
# 📁 train_bert_model.ipynb (Save as Jupyter Notebook)

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import os

# ========== Step 1: Load Preprocessed Dataset ==========
df = pd.read_csv("data/combined_dataset.csv")

# Optional: Reduce size for quicker training
# df = df.sample(10000, random_state=42)

# ========== Step 2: Split into Train/Test ==========
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['content'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

# ========== Step 3: Tokenization ==========
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# ========== Step 4: Create Torch Dataset ==========
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

# ========== Step 5: Load Model ==========
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# ========== Step 6: Training ==========
training_args = TrainingArguments(
    output_dir="model/bert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# ========== Step 7: Save Model and Tokenizer ==========
os.makedirs("model/bert_model", exist_ok=True)
model.save_pretrained("model/bert_model")
tokenizer.save_pretrained("model/bert_model")

print("✅ BERT model and tokenizer saved to model/bert_model/")


FileNotFoundError: [Errno 2] No such file or directory: 'data/combined_dataset.csv'